## Import libraries

In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Data preprocessing

In [89]:
!wget wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv -O data.csv

--2023-10-05 02:42:58--  http://wget/
Resolving wget (wget)... failed: No such host is known. .
wget: unable to resolve host address 'wget'
--2023-10-05 02:43:00--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: 'data.csv'

     0K .......... .......... .......... .......... ..........  3% 1.24M 1s
    50K .......... .......... .......... .......... ..........  6% 4.11M 1s
   100K .......... .......... .......... .......... .......... 10% 2.40M 1s
   150K .......... .......... .......... .......... .......... 13% 8.69M 0s
   200K .......... .......... .......... .......... .......... 17% 7.73M 0s
   250

In [90]:
# Read the data
df = pd.read_csv('data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [91]:
# Keep some columns relevant to this homework
cols = \
    ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders',
    'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']
df = df[cols]

In [92]:
# Rename columns using snake case
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [93]:
# Check missing values
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [94]:
# Fill missing values with 0
df = df.fillna(0)

In [95]:
# Rename MSRP to price
df = df.rename(columns={'msrp': 'price'})

## Question 1

What is the most frequent observation (mode) for the column `transmission_type`?

- `AUTOMATIC`
- `MANUAL`
- `AUTOMATED_MANUAL`
- `DIRECT_DRIVE`

In [96]:
df.transmission_type.value_counts()
# ANSWER: Automatic

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

## Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

- `engine_hp` and `year`
- `engine_hp` and `engine_cylinders`
- `highway_mpg` and `engine_cylinders`
- `highway_mpg` and `city_mpg`


In [97]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[numeric_cols].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [98]:
# Order correlation matrix in pairs in descending order after taking abs value
df[numeric_cols].corr().apply(lambda x: abs(x)).unstack().sort_values(ascending=False).drop_duplicates()
# ANSWER: highway_mpg and city_mpg

year              year                1.000000
highway_mpg       city_mpg            0.886829
engine_hp         engine_cylinders    0.774851
                  price               0.650095
engine_cylinders  highway_mpg         0.614541
                  city_mpg            0.587306
                  price               0.526274
city_mpg          engine_hp           0.424918
highway_mpg       engine_hp           0.415707
year              engine_hp           0.338714
                  highway_mpg         0.258240
                  price               0.227590
city_mpg          year                0.198171
highway_mpg       price               0.160043
city_mpg          price               0.157676
year              engine_cylinders    0.040708
dtype: float64

## Make `price` binary

* Now we need to turn the `price` variable from numeric into a binary format.
* Let's create a variable `above_average` which is `1` if the `price` is above its mean value and `0` otherwise.

## Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value (`above_average`) is not in your dataframe.

In [99]:
# Above average variable for price
above_average = (df.price > df.price.mean()).astype(int)

# Select all columns except price
X = df.drop('price', axis=1)
# Split data into train, validation and test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, above_average, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

## Question 3

* Calculate the mutual information score between `above_average` and other categorical variables in our dataset. 
  Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the lowest mutual information score?
  
- `make`
- `model`
- `transmission_type`
- `vehicle_style`


In [100]:
# Get mutual information scores in a loop
categorical_cols = df.select_dtypes(include=[object]).columns.tolist()
df_mi = X_train[categorical_cols].apply(lambda col: mutual_info_score(col, y_train))
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi
# ANSWER: transmission_type

,MI
model,0.462344
make,0.239769
vehicle_style,0.084143
transmission_type,0.020958


## Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.60
- 0.72
- 0.84
- 0.95

In [101]:
def get_accuracy_score(X_train, X_val, X_test, y_train, y_val, col_remove=None):
    # Remove column if specified
    if col_remove:
        X_train = X_train.drop(col_remove, axis=1)
        X_val = X_val.drop(col_remove, axis=1)
        X_test = X_test.drop(col_remove, axis=1)
    
    # Get categorical columns
    categorical_cols = X_train.select_dtypes(include=[object]).columns.tolist()
    
    # Concatenate datasets for one-hot encoding
    comb_df = pd.concat([X_train, X_val, X_test], axis=0)
    # One-hot encoding for categorical variables
    comb_df_oh = pd.get_dummies(comb_df, columns=categorical_cols, prefix=categorical_cols, dtype=int)

    # Split the encoded data back into train, validation, and test sets
    X_train_oh = comb_df_oh[:len(X_train)]
    X_val_oh = comb_df_oh[len(X_train):len(X_train) + len(X_val)]
    X_test_oh = comb_df_oh[len(X_train) + len(X_val):]

    # Fit logistic regression model
    model = LogisticRegression(solver='liblinear', C=10, max_iter= 1000, random_state=42)
    model.fit(X_train_oh, y_train)
    # Get accuracy on the validation set
    y_pred = model.predict_proba(X_val_oh)[:,1]
    return (y_val.values == (y_pred >= 0.5)).mean().round(6)

all_accuracy = get_accuracy_score(X_train, X_val, X_test, y_train, y_val)
print('Accuracy score:', all_accuracy)

Accuracy score: 0.939572


## Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `year`
- `engine_hp`
- `transmission_type`
- `city_mpg`

> **Note**: the difference doesn't have to be positive

In [102]:
acc_diff = {}
for col in X_train.columns:
    acc_with_col = get_accuracy_score(X_train, X_val, X_test, y_train, y_val, col)
    acc_diff[col] = abs(all_accuracy - acc_with_col)

dict(sorted(acc_diff.items(), key=lambda item: item[1], reverse=True))
# ANSWER: Transmission type


{'model': 0.02014299999999991,
 'engine_hp': 0.015106999999999982,
 'year': 0.008812000000000042,
 'city_mpg': 0.007133999999999974,
 'make': 0.006713999999999998,
 'engine_cylinders': 0.0062950000000000506,
 'transmission_type': 0.005454999999999988,
 'highway_mpg': 0.0025180000000000202,
 'vehicle_style': 0.0008390000000000342}

### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column `price`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver `'sag'`. Set the seed to `42`.
* This model also has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`.
* Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

- 0
- 0.01
- 0.1
- 1
- 10

> **Note**: If there are multiple options, select the smallest `alpha`.

In [ ]:
# Age transformation
df['age'] = 2017 - df.year
df.drop('year', axis=1, inplace=True)

In [111]:
# Get categorical columns
categorical_cols = df.select_dtypes(include=[object]).columns.tolist()
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Set up the column transformer with OneHotEncoder for categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(max_categories=5, drop='if_binary', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'  # let numerical columns pass through without any transformation
)

# Apply the column transformer to the dataframe
encoded_data = preprocessor.fit_transform(df)

# Retrieve the one-hot encoded column names and append the numerical column names
encoded_columns = (preprocessor.named_transformers_['cat']
                    .get_feature_names_out(categorical_cols))\
                    .tolist() + numeric_cols
                    
# Convert the array back to a dataframe with proper column names
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)

In [112]:
# Apply log1p transformation to price
y = encoded_df.price
y_log = np.log1p(y)

# Select all columns except price
X = encoded_df.drop('price', axis=1)
numeric_cols.remove('price')
# Split data into train, validation and test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

In [113]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print('RMSE:', rmse)


RMSE: 0.4938734923689823


In [110]:
# Fit Ridge regression model
for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=alpha, random_state=42, solver='sag')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred, squared=False)
    score = round(score, 3)
    print(f'alpha={alpha}, score={score:.4f}')
    
# ANSWER: alpha=0

c:\Users\aarro\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


alpha=0, score=0.4910


c:\Users\aarro\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


alpha=0.01, score=0.4910


c:\Users\aarro\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


alpha=0.1, score=0.4910


c:\Users\aarro\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


alpha=1, score=0.4910
alpha=10, score=0.4910


c:\Users\aarro\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
